In [ ]:
## Visualize the trained model

import sys
#sys.path.append('deps/sparse_rrt')
sys.path.append('.')
import torch
import torch.nn as nn
import importlib
#from model import ae_s2d
#from model import cvae_s2d_model1 as cvae_s2d

from model.SMPNet import SMPNet
#from tools import data_loader
from tools.utility import *
#from plan_utility import cart_pole, cart_pole_obs, pendulum, acrobot_obs, car_obs
import argparse
import numpy as np
import random
import os
from tqdm import tqdm, trange


In [ ]:
parser = argparse.ArgumentParser()
# for training
parser.add_argument('--model_dir', type=str, default='/ilab/users/ym420/Documents/ylmiao/results/SMPNet/',help='path for saving trained models')
parser.add_argument('--no_env', type=int, default=10,help='directory for obstacle images')
parser.add_argument('--no_motion_paths', type=int,default=200,help='number of optimal paths in each environment')
# Model parameters
parser.add_argument('--e_net_input_size', type=int, default=32, help='dimension of environment encoding network input')
parser.add_argument('--e_net_output_size', type=int, default=28, help='dimension of environment encoding network output')
parser.add_argument('--model_id', type=int, default=0, help='id of the model that we are using')

parser.add_argument('--input_size', type=int, default=2, help='dimension of input to be reconstructed')
parser.add_argument('--latent_size', type=int, default=2, help='dimension of latent variable')
parser.add_argument('--cond_size', type=int, default=32, help='dimension of condition variable (start, goal, environment encoding)')

parser.add_argument('--learning_rate', type=float, default=0.0001)
parser.add_argument('--beta', type=float, default=1.0, help='scale the KL divergence loss relative to reconstruction loss')

parser.add_argument('--lr_schedule_gamma', type=float, default=0.9)
parser.add_argument('--lr_schedule_patience', type=int, default=2)

parser.add_argument('--device', type=int, default=2, help='cuda device')

parser.add_argument('--data_folder', type=str, default='/freespace/local/ym420/course/cse535/data/')

parser.add_argument('--start_epoch', type=int, default=0)
parser.add_argument('--env_type', type=str, default='s2d', help='environment')
parser.add_argument('--world_size', nargs='+', type=float, default=[20.0, 20.0], help='boundary of world')
parser.add_argument('--opt', type=str, default='Adam')

args = parser.parse_args("")


In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(args.device)
# environment setting
cae = None
mlp = None

# load net
# load previously trained model if start epoch > 0

# dynamically import model
ae_module = importlib.import_module('model.ae_%s_model_%d' % (args.env_type, args.model_id))
cvae_module = importlib.import_module('model.cvae_%s_model_%d' % (args.env_type, args.model_id))
e_net = ae_module.Encoder(input_size=args.e_net_input_size, output_size=args.e_net_output_size)
cvae = cvae_module.CVAE(input_size=args.input_size, latent_size=args.latent_size, cond_size=args.cond_size)

data_loader = importlib.import_module('tools.data_loader_%s' % (args.env_type))
plan_util = importlib.import_module('plan_util.%s' % (args.env_type))
normalize = plan_util.normalize

smpnet = SMPNet(e_net, cvae)
model_dir = args.model_dir + "%s/model_%d/lr_%f_lrgamma_%f_lrpatience_%d_opt_%s_beta_%f/" % \
                            (args.env_type, args.model_id, args.learning_rate, \
                             args.lr_schedule_gamma, args.lr_schedule_patience, args.opt, args.beta)
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)
model_path='smpnet_epoch_%d.pkl' %(args.start_epoch)
torch_seed, np_seed, py_seed = 0, 0, 0
if args.start_epoch > 0:
    load_net_state(smpnet, os.path.join(model_dir, model_path))
    torch_seed, np_seed, py_seed = load_seed(os.path.join(model_dir, model_path))
    # set seed after loading
    torch.manual_seed(torch_seed)
    np.random.seed(np_seed)
    random.seed(py_seed)

if torch.cuda.is_available():
    smpnet.cuda()
    smpnet.cvae.cuda()
    smpnet.e_net.cuda()
    if args.opt == 'Adagrad':
        smpnet.set_opt(torch.optim.Adagrad, lr=args.learning_rate)
    elif args.opt == 'Adam':
        smpnet.set_opt(torch.optim.Adam, lr=args.learning_rate)
    elif args.opt == 'SGD':
        smpnet.set_opt(torch.optim.SGD, lr=args.learning_rate, momentum=0.9)
    elif args.opt == 'ASGD':
        smpnet.set_opt(torch.optim.ASGD, lr=args.learning_rate)
if args.start_epoch > 0:
    load_opt_state(smpnet, os.path.join(model_dir, model_path))

# load train and test data
print('loading...')
data_folder = args.data_folder+args.env_type+'/'
waypoint_dataset, start_dataset, goal_dataset, obs, start_indices, goal_indices, env_indices \
         = data_loader.load_train_dataset(N=args.no_env, NP=args.no_motion_paths,
                                            data_folder=data_folder)
start_dataset, goal_dataset = np.array(start_dataset), np.array(goal_dataset)

# randomize the dataset before training
data=list(zip(waypoint_dataset, start_indices, goal_indices, env_indices))
random.shuffle(data)
waypoint_dataset,start_indices,goal_indices,env_indices=list(zip(*data))
waypoint_dataset = list(waypoint_dataset)
start_indices = list(start_indices)
goal_indices = list(goal_indices)
env_indices = list(env_indices)
waypoint_dataset = np.array(waypoint_dataset)
start_indices, goal_indices, env_indices = np.array(start_indices), np.array(goal_indices), np.array(env_indices)

# use 5% as validation dataset
val_len = int(len(waypoint_dataset) * 0.05)
val_waypoint_dataset = waypoint_dataset[-val_len:]
val_start_indices = start_indices[-val_len:]
val_goal_indices = goal_indices[-val_len:]
val_env_indices = env_indices[-val_len:]

waypoint_dataset = waypoint_dataset[:-val_len]
start_indices, goal_indices, env_indices = start_indices[:-val_len], goal_indices[:-val_len], env_indices[:-val_len]
